# PostgreSQL Summary Stats and Window Functions
Here you can access the `summer_medals` table used in the course. To access the table, you will need to specify the `medals` schema in your queries (e.g., `medals.summer_medals`).

In [1]:
-- Add your own queries here
SELECT *
FROM medals.summer_medals
LIMIT 5

,year,city,sport,discipline,athlete,country,gender,event,medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver


## Window functions

Funções que se comportam como o group by, mas cada row permanece no final. Então fica mais fácil iterar sobre valores em cada row.

### _Introdução a window function_

In [4]:
-- COMO ADICIONAR RANK(numeração) NAS LINHAS
SELECT
    year, event, country,
    ROW_NUMBER() OVER() AS row_n
FROM medals.summer_medals
WHERE medal='Gold'
LIMIT 5;

,year,event,country,row_n
0,1896,100M Freestyle,HUN,1
1,1896,100M Freestyle For Sailors,GRE,2
2,1896,1200M Freestyle,HUN,3
3,1896,400M Freestyle,AUT,4
4,1896,100M,USA,5


In [9]:
/* ORDER BY subclausula em OVER para ordenar em função de algum campo especificado*/
SELECT
    year,event,country,
    ROW_NUMBER() OVER(ORDER BY year DESC) AS row_n
FROM medals.summer_medals
WHERE medal='Gold'
LIMIT 5;

/* É possível ordenar por dois campos ao mesmo tempo e em ASC e DESC*/
SELECT
    year,event,country,
    ROW_NUMBER() OVER(ORDER BY year DESC,event ASC) AS row_n
FROM medals.summer_medals
WHERE medal='Gold'
ORDER BY country ASC, year DESC -- ordenando por fora, não influencia row_n
LIMIT 5;

,year,event,country,row_n
0,2012,1500M,,26
1,2012,63KG,,159
2,2012,1500M,ALG,27
3,2000,1500M,ALG,1988
4,1996,1500M,ALG,2653


In [17]:
/* LAG(coluna,n) retorna o valor de n rows anteriores ao current row */
WITH Discus_Gold AS(
SELECT
    year,country AS campeao
FROM medals.summer_medals
WHERE
    year IN ('1996','2000','2004','2008','2012')
    AND gender='Men' AND medal='Gold'
    AND event='Discus Throw') -- Criando CTE para ser usado posteriormente
    
SELECT
    year,campeao,
    LAG(campeao,1) OVER(ORDER BY year ASC) AS ultimo_campeao
FROM Discus_Gold;

-- na saída podemos notas que o ultimo_campeao primeiro row é null, nao pegamos esse valor

,year,campeao,ultimo_campeao
0,1996,GER,None
1,2000,LTU,GER
2,2004,LTU,LTU
3,2008,EST,LTU
4,2012,GER,EST


In [1]:
/* PARTITION BY subclausula em OVER para separar em função de algum campo especificado, e resetado a função aplicada para cada partição*/

WITH Discus_Gold AS(
SELECT
    year,country AS campeao,event
FROM medals.summer_medals
WHERE
    year IN ('1996','2000','2004','2008','2012')
    AND gender='Men' AND medal='Gold'
    AND event='Discus Throw' OR event='Triple Jump') -- Criando CTE para ser usado posteriormente

SELECT
    year,event,campeao,
    LAG(campeao) OVER(PARTITION BY event -- possivel particionar em mais de um
                     ORDER BY event ASC, year ASC) 
FROM Discus_Gold
ORDER BY event ASC, year ASC;

,year,event,campeao,lag
0,1996,Discus Throw,GER,None
1,2000,Discus Throw,LTU,GER
2,2004,Discus Throw,LTU,LTU
3,2008,Discus Throw,EST,LTU
4,2012,Discus Throw,GER,EST
...,...,...,...,...
96,2012,Triple Jump,USA,USA
97,2012,Triple Jump,ITA,USA
98,2012,Triple Jump,KAZ,ITA
99,2012,Triple Jump,COL,KAZ


### _Fetching,ranking and paging_

- _**Fetching:**_
-  **Relativo(relação com o current row):**
1. LAG (n para trás)
2. LEAD (n para frente)
-  **Absoluto(não relativo ao current row):**
1. FIRST_VALUE(coluna)
2. LAST_VALUE(coluna)

In [3]:
/*Usando LEAD*/
WITH cidades AS(
    SELECT DISTINCT year,city
    FROM medals.summer_medals
    WHERE year IN ('1996','2000','2004','2008','2012')
)

SELECT
    year,city,
    LEAD(city,1) OVER (ORDER BY year ASC) AS proxima_cidade,
    LEAD(city,2) OVER (ORDER BY year ASC) AS depois_proxima
FROM cidades;

,year,city,proxima_cidade,depois_proxima
0,1996,Atlanta,Sydney,Athens
1,2000,Sydney,Athens,Beijing
2,2004,Athens,Beijing,London
3,2008,Beijing,London,None
4,2012,London,None,None


In [7]:
/*Usando FIRST ou LAST*/
WITH cidades AS(
    SELECT DISTINCT year,city
    FROM medals.summer_medals
    WHERE year IN ('1996','2000','2004','2008','2012')
)

SELECT
    year,city,
    FIRST_VALUE(city) OVER(ORDER BY year ASC) AS primeira,
    LAST_VALUE(city) OVER(ORDER BY year ASC 
                          RANGE BETWEEN UNBOUNDED PRECEDING AND
                                        UNBOUNDED FOLLOWING)
    AS ultima
FROM cidades;

-- É necessário colocar unbounded para não pegar o valor do current row

,year,city,primeira,ultima
0,1996,Atlanta,Atlanta,London
1,2000,Sydney,Atlanta,London
2,2004,Athens,Atlanta,London
3,2008,Beijing,Atlanta,London
4,2012,London,Atlanta,London


### _Fetching,ranking and paging_

- _**Ranking:**_
1. ROW_NUMBER(sempre adiciona um valor diferente)
2. RANK(adiciona valores iguais se empate, pulando a quantidade de valores empatadas)
3. DENSE_RANK(adiciona valores iguais se empate, não pulando a quantidade de valores empatadas)

In [11]:
WITH paises_jogos AS(
    SELECT DISTINCT country,
    COUNT(DISTINCT year) AS jogos
    FROM medals.summer_medals
    GROUP BY country
)

SELECT country,jogos,
    ROW_NUMBER() OVER(ORDER BY jogos DESC) AS row_n,
    RANK() OVER(ORDER BY jogos DESC) AS rank_n,
    DENSE_RANK() OVER(ORDER BY jogos DESC) AS dense_rank_n
FROM paises_jogos
ORDER BY jogos DESC, country ASC
LIMIT 5;
-- lembrar de particionar na variavel, quando necessário

,country,jogos,row_n,rank_n,dense_rank_n
0,GBR,27,1,1,1
1,DEN,26,4,2,2
2,FRA,26,2,2,2
3,USA,26,3,2,2
4,CAN,25,7,5,3


### _Fetching,ranking and paging_

- _**Paging:**_
1. NTILE(n) divide os dados em n, aproximadamente iguais, páginas
Usado tanto para agilizar transmissão de dados, API, ou para encontrar quantils.

In [12]:
WITH pais_medalhas AS(
    SELECT country, COUNT(*) AS medalhas
    FROM medals.summer_medals
    GROUP BY country
    LIMIT 10
)

SELECT
    country,medalhas,
    NTILE(5) OVER(ORDER BY medalhas DESC) AS pagina
FROM pais_medalhas

,country,medalhas,pagina
0,HUN,1079,1
1,NED,851,1
2,DEN,507,2
3,CUB,410,2
4,SUI,380,3
5,AUT,146,3
6,ALG,15,4
7,VEN,12,4
8,SIN,4,5
9,UAE,1,5


In [14]:
/*Usando para encontrar quantile 33%*/
WITH pais_medalhas AS(
    SELECT country, COUNT(*) AS medalhas
    FROM medals.summer_medals
    GROUP BY country
)

SELECT
    country,medalhas,
    NTILE(3) OVER(ORDER BY medalhas DESC) -- corta nos 33% maiores medalhistas
        AS primeiros_33
FROM pais_medalhas
LIMIT 5;

,country,medalhas,primeiros_33
0,USA,4585,1
1,URS,2049,1
2,GBR,1720,1
3,FRA,1396,1
4,GER,1305,1


### _Aggregate window functions and Frames_
SOMA E MAXIMO

In [17]:
/*MAX como função de maximo cumulativo*/
WITH Brasil_ouro AS(
    SELECT year,
        COUNT(*) AS medalhas
    FROM medals.summer_medals
    WHERE
        country='BRA' AND medal='Gold' AND year>=1992
    GROUP BY year
    ORDER BY year ASC
)

SELECT
    year,medalhas,
    MAX(medalhas) OVER (ORDER BY year ASC) AS max_medalhas
FROM Brasil_ouro;

,year,medalhas,max_medalhas
0,1992,13,13
1,1996,5,13
2,2004,18,18
3,2008,14,18
4,2012,14,18


In [19]:
/*SUM como função de soma cumulativa*/
WITH Brasil_ouro AS(
    SELECT year,
        COUNT(*) AS medalhas
    FROM medals.summer_medals
    WHERE
        country='BRA' AND medal='Gold' AND year>=1992
    GROUP BY year
    ORDER BY year ASC
)

SELECT
    year,medalhas,
    SUM(medalhas) OVER (ORDER BY year ASC) AS soma_medalhas
FROM Brasil_ouro;
-- Usar PARTITION BY quando necessário. Exp: mais de um país

,year,medalhas,soma_medalhas
0,1992,13,13
1,1996,5,18
2,2004,18,36
3,2008,14,50
4,2012,14,64


### _Aggregate window functions and Frames_
Frames: 
 EX: frame = RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
por default  o frame é do começo da tabela até o current row.

 - ROWS BETWEEN [COMECO] AND [FIM].
1.  n PRECEDING (n valores atras)
2.  n FOLLOWING (n valores a frente)
3.  CURRENT ROW (row que está)

In [21]:
/*MAX como função de maximo cumulativo*/
WITH Brasil_ouro AS(
    SELECT year,
        COUNT(*) AS medalhas
    FROM medals.summer_medals
    WHERE
        country='BRA' AND medal='Gold' AND year>=1992
    GROUP BY year
    ORDER BY year ASC
)

SELECT
    year,medalhas,
    MAX(medalhas) OVER (ORDER BY year ASC
                    ROWS BETWEEN 1 PRECEDING -- Compara o MAX com 1 valor antes
                    AND CURRENT ROW) AS max_medalhas
FROM Brasil_ouro;

,year,medalhas,max_medalhas
0,1992,13,13
1,1996,5,13
2,2004,18,18
3,2008,14,18
4,2012,14,14


### _Aggregate window functions and Frames_
- Moving Average:
Média dos ultimos n períodos, em relação ao atual.
- Moving Total:
Total dos ultimos n períodos, em relação ao atual.

In [30]:
-- MA
WITH EUA_ouro AS(
    SELECT year,
        COUNT(*) AS medalhas
    FROM medals.summer_medals
    WHERE
        country='USA' AND medal='Gold' AND year>=1980
    GROUP BY year
    ORDER BY year ASC
)

SELECT
    year, medalhas,
    ROUND(AVG(medalhas) OVER(ORDER BY year ASC
                       ROWS BETWEEN 
                       2 PRECEDING AND CURRENT ROW),2) AS medalhas_MA,
    ROUND(SUM(medalhas) OVER(ORDER BY year ASC
                       ROWS BETWEEN 
                       UNBOUNDED PRECEDING AND CURRENT ROW),2) AS ROWS_RT,
    ROUND(SUM(medalhas) OVER(ORDER BY year ASC
                       RANGE BETWEEN
                       UNBOUNDED PRECEDING AND CURRENT ROW),2) AS RANGE_RT
FROM EUA_ouro
ORDER BY year ASC
LIMIT 5;

-- RANGE trata a soma do ORDER BY variavel dando mesmos valores(soma e dá um valor só) quando a variavel é igual nas linhas

,year,medalhas,medalhas_ma,rows_rt,range_rt
0,1984,168,168.00,168.0,168.0
1,1988,77,122.50,245.0,245.0
2,1992,89,111.33,334.0,334.0
3,1996,160,108.67,494.0,494.0
4,2000,130,126.33,624.0,624.0


### **Beyond Window functions**

In [34]:
-- tabela que sera usada para pivotar
WITH country_awards AS(
    SELECT country, year,
        COUNT(*) AS awards
    FROM medals.summer_medals
    WHERE
        country IN ('CHN','RUS','USA')
        AND year BETWEEN 2004 AND 2012
        AND medal='Gold' AND sport='Gymnastics'
    GROUP BY country,year
    ORDER BY country ASC, year ASC
)

SELECT country, year,
    RANK() OVER(PARTITION BY year ORDER BY awards DESC)::INTEGER
    AS rank
FROM country_awards
ORDER BY country ASC, year ASC
LIMIT 5;

,country,year,rank
0,CHN,2004,3
1,CHN,2008,1
2,CHN,2012,1
3,RUS,2004,1
4,RUS,2008,2


In [ ]:
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
-- alocar a tabela pivo                       
$$) AS ct(country VARCHAR,
          "2004" INTEGER,
          "2008" INTEGER,
          "2012" INTEGER)
ORDER BY country ASC;

Error: CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
-- alocar a tabela pivo                       
$$) AS ct(country VARCHAR,
          "2004" INTEGER,
          "2008" INTEGER,
          "2012" INTEGER)
ORDER BY country ASC; - permission denied to create extension "tablefunc"

In [36]:
/* ROLLUP formas de agregar com soma total*/
SELECT country,medal,COUNT(*) AS AWARDS
FROM medals.summer_medals
WHERE
    year = 2008 AND country IN ('CHN','RUS')
GROUP BY ROLLUP(country,medal)
ORDER BY country ASC,medal ASC;
-- TOTAL APENAS DA PRIMEIRA ENTRADA country

,country,medal,awards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,CHN,None,184
4,RUS,Bronze,56
5,RUS,Gold,43
6,RUS,Silver,44
7,RUS,None,143
8,None,None,327


In [37]:
/* CUBE formas de agregar com soma total*/
SELECT country,medal,COUNT(*) AS AWARDS
FROM medals.summer_medals
WHERE
    year = 2008 AND country IN ('CHN','RUS')
GROUP BY CUBE(country,medal)
ORDER BY country ASC,medal ASC;
-- TOTAL DE TODAS AS ENTRADAS country,medal

,country,medal,awards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,CHN,None,184
4,RUS,Bronze,56
5,RUS,Gold,43
6,RUS,Silver,44
7,RUS,None,143
8,None,Bronze,113
9,None,Gold,117


In [40]:
/* COALESCE para substituir valores*/
SELECT COALESCE(country,'Ambos paises') AS country,
       COALESCE(medal,'Total medalhas') AS medalha,
       COUNT(*) AS AWARDS
FROM medals.summer_medals
WHERE
    year = 2008 AND country IN ('CHN','RUS')
GROUP BY CUBE(country,medal)
ORDER BY country ASC,medal ASC;
-- STRING_AGG(valor, separador = ',') transforma coluna em linha com separador

,country,medalha,awards
0,Ambos paises,Bronze,113
1,Ambos paises,Gold,117
2,Ambos paises,Silver,97
3,Ambos paises,Total medalhas,327
4,CHN,Bronze,57
5,CHN,Gold,74
6,CHN,Silver,53
7,CHN,Total medalhas,184
8,RUS,Bronze,56
9,RUS,Gold,43
